In [1]:
import os
import numpy as np
import cv2
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *

In [2]:
# ----------------------------------------load data module begin-----------------------------------------------------
def load_path(data_set, root_dir = 'B:/MURA/MURA-v1.1/', data_type = 'XR_ELBOW'):
    # load MURA data and return data path list and label numpy array
    data_path = root_dir + data_set + '/' + data_type + '/'
    Path = []
    labels = []
    for root, dirs, files in os.walk(data_path):  # read all images, os.walk returns iterators genertor traverses all files
        for name in files:
            if root.split('_')[-1] == 'positive':  # positive label == 1；
                path_1 = os.path.join(root, name)
                Path.append(path_1)
                labels += [1]
            elif root.split('_')[-1] == 'negative':  # negative label == 1；
                path_1 = os.path.join(root, name)
                Path.append(path_1)
                labels += [0]
            else:
                continue
    labels = np.asarray(labels)
    return Path, labels

#  path list in, image list out
def load_image(Path='../valid', imsize=224):
    Images = []
    i = 1
    for path in Path:

        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            print("null")
            continue
        else:
            if i % 500 == 0:
                print ("loading image ", i)
            
            image = cv2.resize(image, (imsize, imsize))   # resize images
            image = randome_rotation_flip(image, imsize)  # predeal images
            Images.append(image)
            i = i+1

    Images = np.asarray(Images).astype('float32')

    # normalization
    print("Starting normalization.......")
    mean = np.mean(Images[:, :, :])
    std = np.std(Images[:, :, :])
    Images[:, :, :] = (Images[:, :, :] - mean) / std
    print("normalization finished")

    Images = np.expand_dims(Images, axis=1)

    return Images

# inner function used by load_image
def randome_rotation_flip(image, imsize=224):
    if random.randint(0, 1):
        image = cv2.flip(image, 1)

    if random.randint(0, 1):
        angle = random.randint(-30, 30)
        M = cv2.getRotationMatrix2D((imsize / 2, imsize / 2), angle, 1)
        # third parameter: the transformed image size
        image = cv2.warpAffine(image, M, (imsize, imsize))
    return image

# ----------------------------------------load data module end-----------------------------------------------------

In [9]:
# -----------------------------------network hyperparameter begin--------------------------------
im_size = 224  # input image size
batch_size = 20 
epochs = 120
x_shape  = (batch_size, 1, 224, 224)
model_save = True
model_reload = False
drop_out_rate = 0.2
my_init = keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

# -----------------------------------network hyperparameter end--------------------------------

In [4]:
def DenseNet(x):
    x1 = Conv2D(16, (3,  3), activation='selu', padding='same', strides=(1, 1), data_format='channels_first')(x)
    x1 = Activation('relu')(x1)
    x1 = concatenate([x1, x] , axis=1)                    
                 
    x2 = Conv2D(16, (3,  3), activation='relu', padding='same', strides=(1, 1), data_format='channels_first')(x1)
    x2 = Activation('relu')(x2)
    x2 = concatenate([x2, x1, x] , axis=1) 

    x3 = Conv2D(32, (3,  3), activation='relu', padding='same', strides=(1, 1), data_format='channels_first')(x2)
    x3 = Activation('relu')(x3)
    x3 = concatenate([x3, x2, x1, x] , axis=1)

    x4 = Conv2D(32, (3,  3), activation='relu', padding='same', strides=(1, 1), data_format='channels_first')(x3)
    x4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(x4)
    return x4

In [6]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

inputs=Input(shape=(1, im_size, im_size))
#224
x = DenseNet(inputs)
x = BatchNormalization(axis=1)(x)
x = Dropout(drop_out_rate)(x)
#112
x = DenseNet(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(drop_out_rate)(x)
#56
x = DenseNet(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(drop_out_rate)(x)
#28
x = Conv2D(64, (1,  1), activation='relu', padding='same', strides=(1, 1), data_format='channels_first')(x)
x = MaxPooling2D(pool_size=(28, 28), data_format='channels_first')(x)

x = Flatten()(x)
x = Dropout(drop_out_rate)(x)
x = Dense(56, activation='relu')(x)

x = Dropout(drop_out_rate)(x)
x = Dense(1, activation='sigmoid')(x)

if model_reload:
    model = keras.models.load_model('Dense_net_7_24.h5')
else:
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer=keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1, 224, 224) 0                                            
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 16, 224, 224) 160         input_2[0][0]                    
__________________________________________________________________________________________________
activation_9 (Activation)       (None, 16, 224, 224) 0           conv2d_13[0][0]                  
__________________________________________________________________________________________________
concatenate_9 (Concatenate)     (None, 17, 224, 224) 0           activation_9[0][0]               
                                                                 input_2[0][0]                

In [7]:
# -------------------------------load train data module begin-----------------------------

# 训练数据
im_size = 224
x_train_path, y_train = load_path(data_set='train')
x_train = load_image(x_train_path, imsize=im_size)
print(x_train.shape)
print(y_train.shape)
  
# -------------------------------load train data module end-----------------------------

loading image  500
loading image  1000
loading image  1500
loading image  2000
loading image  2500
loading image  3000
loading image  3500
loading image  4000
loading image  4500
Starting normalization.......
normalization finished
(4931, 1, 224, 224)
(4931,)


In [10]:
# 开始训练
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)  

Train on 4437 samples, validate on 494 samples
Epoch 1/120
4437/4437 [==============================] - 119s 27ms/sample - loss: 0.8263 - accuracy: 0.5292 - val_loss: 0.7612 - val_accuracy: 0.1498
Epoch 2/120
4437/4437 [==============================] - 111s 25ms/sample - loss: 0.6988 - accuracy: 0.5391 - val_loss: 0.6370 - val_accuracy: 0.7935
Epoch 3/120
4437/4437 [==============================] - 111s 25ms/sample - loss: 0.6850 - accuracy: 0.5506 - val_loss: 0.6847 - val_accuracy: 0.7004
Epoch 4/120
4437/4437 [==============================] - 111s 25ms/sample - loss: 0.6774 - accuracy: 0.5711 - val_loss: 0.6230 - val_accuracy: 0.8198
Epoch 5/120
4437/4437 [==============================] - 111s 25ms/sample - loss: 0.6754 - accuracy: 0.5756 - val_loss: 0.6119 - val_accuracy: 0.8623
Epoch 6/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.6714 - accuracy: 0.5876 - val_loss: 0.6615 - val_accuracy: 0.3826
Epoch 7/120
4437/4437 [==============================

4437/4437 [==============================] - 110s 25ms/sample - loss: 0.4734 - accuracy: 0.7832 - val_loss: 0.5821 - val_accuracy: 0.8239
Epoch 56/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.4649 - accuracy: 0.7872 - val_loss: 0.2888 - val_accuracy: 0.9696
Epoch 57/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.4622 - accuracy: 0.7877 - val_loss: 0.4088 - val_accuracy: 0.9008
Epoch 58/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.4628 - accuracy: 0.7868 - val_loss: 0.4345 - val_accuracy: 0.9332
Epoch 59/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.4501 - accuracy: 0.7963 - val_loss: 0.5098 - val_accuracy: 0.8988
Epoch 60/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.4513 - accuracy: 0.7965 - val_loss: 0.5513 - val_accuracy: 0.8623
Epoch 61/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.4499 - accuracy: 0.798

4437/4437 [==============================] - 110s 25ms/sample - loss: 0.2903 - accuracy: 0.8702 - val_loss: 0.7539 - val_accuracy: 0.7794
Epoch 110/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.2963 - accuracy: 0.8749 - val_loss: 0.5895 - val_accuracy: 0.8603
Epoch 111/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.2743 - accuracy: 0.8772 - val_loss: 0.6820 - val_accuracy: 0.8036
Epoch 112/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.2837 - accuracy: 0.8812 - val_loss: 0.8023 - val_accuracy: 0.7814
Epoch 113/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.3004 - accuracy: 0.8713 - val_loss: 0.7314 - val_accuracy: 0.7874
Epoch 114/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.2768 - accuracy: 0.8837 - val_loss: 0.4375 - val_accuracy: 0.8907
Epoch 115/120
4437/4437 [==============================] - 110s 25ms/sample - loss: 0.2767 - accuracy:

In [12]:
# test
x_test_path, y_test = load_path(data_set='valid')
x_test = load_image(x_test_path, im_size)

Starting normalization.......
normalization finished
465/465 [==============================] - 3s 6ms/sample - loss: 0.7240 - accuracy: 0.7505


In [14]:
res = model.evaluate(x_test, y_test)  

465/465 [==============================] - 3s 7ms/sample - loss: 0.7240 - accuracy: 0.7505


In [15]:
if model_save:
    model.save('Dense_net_7_24.h5')